# Optimize QAOA parameters for LABS

We'll show an example for overlap, but optimizing the expectation is analogous

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from importlib_resources import files

from qokit.qaoa_objective_labs import get_qaoa_labs_objective
from qokit.parameter_utils import (
    from_fourier_basis,
    to_fourier_basis,
    extrapolate_parameters_in_fourier_basis,
)

In [2]:
# get best known parameters
df = pd.read_json(
    files("qokit.assets").joinpath("best_known_QAOA_parameters_wrt_overlap.json"),
    orient='index'
)

In [3]:
N = 10

In [4]:
p_known = df[df['N'] == N]['p'].max()
print(f"Maximum p available for N={N} is {p_known}")
row = df[(df['N'] == N) & (df['p'] == p_known)].squeeze()
print(f"Success probability at p={p_known} is {row['overlap']}")

Maximum p available for N=10 is 8
Success probability at p=8 is 0.5028762135


We will try optimizing further!

Specifically, we will extrapolate the parameters in the fourier basis using the interpolation scheme of https://arxiv.org/abs/1812.01041 and then optimize in the fourier basis

In [5]:
p = p_known+1

beta = row['beta']
gamma = row['gamma']

u, v = to_fourier_basis(gamma, beta)
init_u, init_v = extrapolate_parameters_in_fourier_basis(u, v, p, 1)
init_freq = np.hstack([init_u, init_v])


f = get_qaoa_labs_objective(N, p, parameterization="freq", objective='overlap')

print(f"Success probability at p={p} before optimization is {1-f(init_freq)}")
res = minimize(f, init_freq, method='COBYLA', options={'rhobeg':0.01 / N})

/opt/omniai/work/instance1/jupyter/jpmc-argonne-quantum-optimization/qokit/qaoa_objective.py:63: UserWarning: CUDA is NOT available, using cfurx simulator
  warnings.warn(f"CUDA is NOT available, using {simulator} simulator")


Success probability at p=9 before optimization is 0.49333386759027675


In [6]:
beta, gamma = from_fourier_basis(res.x[:p], res.x[p:])
f = get_qaoa_labs_objective(N, p, parameterization="gamma beta", objective='overlap')

print(f"Success probability at p={p} after optimization is {1-f(gamma, beta)}")

Success probability at p=9 after optimization is 0.5243661480217591


/opt/omniai/work/instance1/jupyter/jpmc-argonne-quantum-optimization/qokit/qaoa_objective.py:63: UserWarning: CUDA is NOT available, using cfurx simulator
  warnings.warn(f"CUDA is NOT available, using {simulator} simulator")
